<a href="https://colab.research.google.com/github/devadityaojha/PoachDetect/blob/master/PoacherDetect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Uncomment the line corresponding to your "runtime type" to run in Google Colab

# CPU:
# !pip install pydub torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# GPU:
# !pip install pydub torch==1.7.0+cu101 torchvision==0.8.1+cu101 torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio
import sys

import matplotlib.pyplot as plt
import IPython.display as ipd
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, Subset, DataLoader


import tqdm

import glob
from scipy.io import wavfile
import matplotlib.pyplot as plt
from scipy import signal
import torch

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
class AudioData(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.landmarks_frame = pd.read_csv(root_dir + csv_file)
        self.root_dir = root_dir
        self.cache = {}

    def __len__(self):
        return len(self.landmarks_frame)

    def transform(self, idx, filepath):
        if idx in self.cache:
          return self.cache[idx]
        if self.landmarks_frame.iloc[idx,4]:
          samplerate, data = wavfile.read(self.root_dir + "gunshots/" + filepath)
        else:
          samplerate, data = wavfile.read(self.root_dir + "not_gunshots/" + filepath)
        data = data[:50000]
        frequencies, times, spectogram = signal.spectrogram(data, samplerate, nfft=1000, mode='angle')

        self.cache[idx] = frequencies[:100]
        return frequencies[:100]

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        filepath = self.landmarks_frame.iloc[idx, 3]

        return {"sample": self.transform(idx, filepath), "gunshot": self.landmarks_frame.iloc[idx, 4]}


audio_dataset = AudioData("audio_labels.csv", "/content/drive/MyDrive/Datasets/")

In [77]:
idxs = np.arange(len(audio_dataset))
np.random.shuffle(idxs)
train, test = idxs[:len(audio_dataset) * 4 // 5], idxs[len(audio_dataset) * 4 // 5:]
train = Subset(audio_dataset, train)
test = Subset(audio_dataset, test)

In [78]:
train_dataloader = DataLoader(train, batch_size=128, shuffle=True)
test_dataloader = DataLoader(test, batch_size=128, shuffle=True)

In [139]:
from re import M
import torch.nn.functional as F

class Model(nn.Module):
    
    def __init__(self, n_input=100):
        super(Model, self).__init__()
        n_channel = 1
        

        self.encoder = nn.Sequential(
            nn.Conv1d(n_input, n_channel, kernel_size=1, stride=2),
            nn.BatchNorm1d(n_channel),
            nn.ReLU(),
            nn.MaxPool1d(1),
            nn.Conv1d(n_channel, n_channel, kernel_size=1),
            nn.BatchNorm1d(n_channel),
            nn.ReLU(),
            nn.MaxPool1d(1),
            nn.Conv1d(n_channel, 2 * n_channel, kernel_size=1),
            nn.BatchNorm1d(2 * n_channel),
            nn.ReLU(),
            nn.MaxPool1d(1)
        )

        self.fc1 = nn.Linear(2 * n_channel, 1)
    
    def forward(self, x):

        x = self.encoder(x)
        x = F.avg_pool1d(x, x.shape[-1])
        x = x.permute(0, 2, 1)
        x = self.fc1(x)

        return torch.sigmoid(x).squeeze()
    
model = Model(n_input=100)
model.to(device)

Model(
  (encoder): Sequential(
    (0): Conv1d(100, 1, kernel_size=(1,), stride=(2,))
    (1): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
    (4): Conv1d(1, 1, kernel_size=(1,), stride=(1,))
    (5): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
    (8): Conv1d(1, 2, kernel_size=(1,), stride=(1,))
    (9): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=2, out_features=1, bias=True)
)

In [140]:
criterion = nn.MSELoss()

In [141]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters())

In [144]:
epochs = 100

for epoch in (range(epochs)):
    total_loss = 0
    for batch in tqdm.notebook.tqdm(train_dataloader, unit='batch'):
        X_batch, y_batch = batch["sample"], batch["gunshot"]
        
        # THESE 5 LINES ARE IMPORTANT, UNDERSTAND WHAT IS HAPPENING HERE
        optimizer.zero_grad()
        a, b = X_batch.size()
        predicted_batch = model(X_batch.to(device).float().view(a, b, 1))
        loss = criterion(predicted_batch.to(device), y_batch.to(device).float())
        loss.backward()
        optimizer.step()
        
        total_loss += loss
        
    print("Epoch {0}: {1}".format(epoch, total_loss))
    if epoch%5 == 0 and epoch!= 0:
        test_batch = next(iter(test_dataloader))
        a, b = test_batch['sample'].size()
        X_test, y_test = test_batch["sample"].to(device).view(a, b, 1), test_batch["gunshot"].to(device)
        predicted = model(X_test.float())
        test_acc = torch.sum(y_test == (predicted >= 0.5), dtype=torch.double) / len(y_test)
        print("\tTest Accuracy {0}".format(test_acc))

  0%|          | 0/10 [00:00<?, ?batch/s]

Epoch 0: 1.7604057788848877


  0%|          | 0/10 [00:00<?, ?batch/s]

Epoch 1: 1.7514060735702515


  0%|          | 0/10 [00:00<?, ?batch/s]

...

Epoch 97: 1.1152348518371582


  0%|          | 0/10 [00:00<?, ?batch/s]

Epoch 98: 1.1259307861328125


  0%|          | 0/10 [00:00<?, ?batch/s]

Epoch 99: 1.1205531358718872


In [145]:
test_batch = next(iter(test_dataloader))
a, b = test_batch['sample'].size()
X_test, y_test = test_batch["sample"].to(device).view(a, b, 1), test_batch["gunshot"].to(device)
predicted = model(X_test.float())
print((predicted >= 0.5).float())
# print(torch.argmax(predicted, dim=1))
test_accuracy = torch.sum(y_test == (predicted >= 0.5), dtype=torch.double) / len(y_test)
print ("Test Accuracy {0}".format(test_accuracy))

tensor([1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0.,
        0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0.,
        0., 1., 1., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1.,
        0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0.,
        1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0.,
        1., 1., 0., 0., 1., 1., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1., 1., 0.,
        0., 0.], device='cuda:0')
Test Accuracy 0.8828125


In [147]:
torch.save(model.state_dict(), 'model_parameters.tar')